In [1]:
import VentricularImage
import VentricularInterpolator
import os
import os.path

reload(VentricularImage)
reload(VentricularInterpolator)

<module 'VentricularInterpolator' from 'VentricularInterpolator.pyc'>

In [2]:
path = os.path.join('C:/Users/guill/Documents/Git/pyqcm/','Layer_10_percent_colored.vtk')

In [3]:
MRI = VentricularImage.VentricularQCM(path,1,41)

AttributeError: 'VentricularQCM' object has no attribute 'dtype'